In [39]:
from datetime import datetime
from elasticsearch import Elasticsearch
from nytimesarticle import articleAPI
import requests
import json


In [3]:
es_url = 'http://192.168.0.26:9200'
api_key = 'jTrPGAlMY9RoTKt6FJKYunrC701lYPu9'

# API
#     Structure     : /articlesearch.json?q={query}&fq={filter}
#     Documentation : https://developer.nytimes.com/docs/articlesearch-product/1/routes/articlesearch.json/get
#     Example       : https://api.nytimes.com/svc/search/v2/articlesearch.json?q=election&api-key=yourkey
#                   : https://api.nytimes.com/svc/search/v2/articlesearch.json?q=trump&api-key=jTrPGAlMY9RoTKt6FJKYunrC701lYPu9
#                   : https://api.nytimes.com/svc/search/v2/articlesearch.json?q=trump&api-key=jTrPGAlMY9RoTKt6FJKYunrC701lYPu9&fq=pub_date:2016-01-01

api_url = ''

document_type = 'news'
index_name = 'NYT'
reload = True

es = Elasticsearch(hosts=es_url)

if reload:
    es.indices.delete(index=index_name, ignore=[400, 404])

In [12]:
index_name = "tweets"
document_type = "tweets"

query_body = {
  "size": 0, 
  "aggs": {
    "group_by_tweets": {
      "terms": {
        "field": "tweet_id.keyword"
      }
    }
  }
}

res = es.search(index=index_name, body=query_body)
print (json.dumps(res,  indent=4))

# total records : print (res['hits']['total']['value'])


{
    "took": 0,
    "timed_out": false,
    "_shards": {
        "total": 1,
        "successful": 1,
        "skipped": 0,
        "failed": 0
    },
    "hits": {
        "total": {
            "value": 7375,
            "relation": "eq"
        },
        "max_score": null,
        "hits": []
    },
    "aggregations": {
        "group_by_tweets": {
            "doc_count_error_upper_bound": 0,
            "sum_other_doc_count": 6354,
            "buckets": [
                {
                    "key": "6.54E+17",
                    "doc_count": 128
                },
                {
                    "key": "7.89E+17",
                    "doc_count": 108
                },
                {
                    "key": "6.25E+17",
                    "doc_count": 105
                },
                {
                    "key": "6.24E+17",
                    "doc_count": 103
                },
                {
                    "key": "6.60E+17",
                    "do

In [17]:
top_activity = {
  "size": 0, 
  "aggs": {
    "group_by_tweets": {
      "date_histogram": {
        "field": "timestamp",
        "interval": "month"
      },
      "aggs": {
        "total_tweets": {
          "value_count": {
            "field": "timestamp"
          }
        },
        "sales_bucket_sort" : {
          "bucket_sort": {
            "sort": [{"total_tweets" : {"order": "desc"} }],
            "size": 5
          }
          
        }
      }
    }
  }
}

res = es.search(index=index_name, body=top_activity)
print (json.dumps(res,  indent=4))

{
    "took": 2,
    "timed_out": false,
    "_shards": {
        "total": 1,
        "successful": 1,
        "skipped": 0,
        "failed": 0
    },
    "hits": {
        "total": {
            "value": 7375,
            "relation": "eq"
        },
        "max_score": null,
        "hits": []
    },
    "aggregations": {
        "group_by_tweets": {
            "buckets": [
                {
                    "key_as_string": "2015-10-01T00:00:00.000Z",
                    "key": 1443657600000,
                    "doc_count": 815,
                    "total_tweets": {
                        "value": 815
                    }
                },
                {
                    "key_as_string": "2015-11-01T00:00:00.000Z",
                    "key": 1446336000000,
                    "doc_count": 599,
                    "total_tweets": {
                        "value": 599
                    }
                },
                {
                    "key_as_string": "2015-

In [28]:
for record in res['aggregations']['group_by_tweets']['buckets']:
    print (f'Date : {record["key_as_string"]}, total of records : {record["doc_count"]}')

Date : 2015-10-01T00:00:00.000Z, total of records : 815
Date : 2015-11-01T00:00:00.000Z, total of records : 599
Date : 2015-12-01T00:00:00.000Z, total of records : 579
Date : 2015-07-01T00:00:00.000Z, total of records : 534
Date : 2016-10-01T00:00:00.000Z, total of records : 532


In [38]:
params = {
    'api-key' : api_key,
    'q' : 'trump',
    'pub_date' : '2016-01-01'    
}

api_url = 'https://api.nytimes.com/svc/search/v2/articlesearch.json'

response = requests.get(url = api_url, params = params)

if response.status_code == 200:    
    print ('ok')
else: 
    print ('error')
    
print (response)  
print (params)


error
<Response [401]>
{'api_key': 'jTrPGAlMY9RoTKt6FJKYunrC701lYPu9', 'q': 'trump', 'pub_date': '2016-01-01'}


In [54]:

api = articleAPI(api_key)

articles = api.search(q='trump', begin_date=20190101, end_date=20190101, page=1)

print (json.dumps(articles['response'],  indent=4))

{
    "docs": [
        {
            "web_url": "https://www.nytimes.com/2019/01/01/us/politics/joe-biden-presidential-campaign-2020.html",
            "snippet": "A series of careful financial decisions, and the creation of nonprofits and academic centers staffed by close advisers, would help a campaign-in-waiting.",
            "lead_paragraph": "When officials at the University of Utah invited Joseph R. Biden Jr. to speak there in December, Mr. Biden\u2019s representatives listed a number of requirements for the appearance. His booking firm, Creative Artists Agency, said the school would need to fly Mr. Biden and his aides to Salt Lake City by private plane. It would buy 1,000 copies of his recent memoir from a designated vendor. There would be no insertion of the word \u201cformer\u201d before \u201cvice president\u201d in social media promotions. And the speaking fee would be $100,000 \u2014 \u201ca reduced rate,\u201d it was explained, for colleges and universities.",
          

In [58]:
document_type = 'news'
index_name = 'nyt'

res = es.index(index=index_name, doc_type=document_type, body=articles)
print (res)

{'_index': 'nyt', '_type': 'news', '_id': 'ZEQpwWoB2K0Y8-is7fUl', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 0, '_primary_term': 1}
